<a href="https://colab.research.google.com/github/FafiCohen/Python_Course/blob/main/document_scanner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!sudo apt install tesseract-ocr
!pip install pytesseract
!pip install -q streamlit
!pip install -q  googletrans
!pip install -q deep_translator

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 35 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 0s (9,708 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debc

In [28]:
%%writefile app.py
import pytesseract
from googletrans import Translator
from deep_translator import GoogleTranslator
from PIL import Image
import streamlit as st
import numpy as np
import cv2
from google.colab.patches import cv2_imshow

def apply_morphological_operation(image):
    # Apply morphological operations (e.g., erosion, dilation)
    morph_kernel = np.ones((5,5), np.uint8)
    processed_image = cv2.morphologyEx(image, cv2.MORPH_CLOSE, morph_kernel, iterations=4)
    return processed_image

def detect_edge_contours(image):
    mask = np.zeros(image.shape[:2], np.uint8)
    bgd_model = np.zeros((1,65), np.float64)
    fgd_model = np.zeros((1,65), np.float64)
    rect = (20, 20, image.shape[1]-20, image.shape[0]-20)
    cv2.grabCut(image, mask, rect, bgd_model, fgd_model, 5, cv2.GC_INIT_WITH_RECT)
    mask2 = np.where((mask==2) | (mask==0), 0, 1).astype('uint8')
    image = image * mask2[:, :, np.newaxis]

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (11, 11), 0)
    canny = cv2.Canny(gray, 0, 200)
    canny = cv2.dilate(canny, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5)))
    contours, hierarchy = cv2.findContours(canny, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
    page = sorted(contours, key=cv2.contourArea, reverse=True)[:5]
    con = np.zeros_like(image)
    con = cv2.drawContours(con, page, -1, (100, 255, 100), 4)
    return con, page

def arrange_points(pts):
    rect = np.zeros((4, 2), dtype='float32')
    pts = np.array(pts)
    s = pts.sum(axis=1)
    rect[0] = pts[np.argmin(s)]
    rect[2] = pts[np.argmax(s)]

    diff = np.diff(pts, axis=1)
    rect[1] = pts[np.argmin(diff)]
    rect[3] = pts[np.argmax(diff)]
    return rect.astype('int').tolist()

def find_corner_points(image):
    page = detect_edge_contours(image)[1]
    con = np.zeros_like(image)
    for c in page:
        epsilon = 0.02 * cv2.arcLength(c, True)
        corners = cv2.approxPolyDP(c, epsilon, True)
        if len(corners) == 4:
            break
    cv2.drawContours(con, c, -1, (100, 255, 100), 4)
    cv2.drawContours(con, corners, -1, (0, 255, 0), 10)
    corners = sorted(np.concatenate(corners).tolist())
    for index, c in enumerate(corners):
        character = chr(65 + index)
        cv2.putText(con, character, tuple(c), cv2.FONT_HERSHEY_SIMPLEX, 5, (255, 0, 0), 5, cv2.LINE_AA)
    corners = arrange_points(corners)
    return con, corners

def find_destination(pts):
    (tl, tr, br, bl) = pts
    widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
    widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
    maxWidth = max(int(widthA), int(widthB))

    heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
    heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
    maxHeight = max(int(heightA), int(heightB))
    destination_corners = [[0, 0], [maxWidth, 0], [maxWidth, maxHeight], [0, maxHeight]]
    return arrange_points(destination_corners)

def apply_perspective_transform(image):
    pts = find_corner_points(image)[1]
    destination_corners = find_destination(pts)
    M = cv2.getPerspectiveTransform(np.float32(pts), np.float32(destination_corners))
    final = cv2.warpPerspective(image, M, (destination_corners[2][0], destination_corners[2][1]), flags=cv2.INTER_LINEAR)
    return final


def preprocess_image(image ):

    gray = cv2.cvtColor(np.array(image), cv2.COLOR_BGR2GRAY)

    # Apply thresholding to binarize the image
    _, binary = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)

    return binary

def perform_ocr_and_translate(image):
    # Preprocess the image
    preprocessed_image = preprocess_image(image)

    # Perform OCR on the preprocessed image
    text = pytesseract.image_to_string(preprocessed_image)
    corrected_text = text.replace("|", "I")
    translated = GoogleTranslator(source='auto', target='iw').translate(corrected_text)
    return translated

def process_image(image, selected_option):
    if selected_option == 'Morphological Operation':
        return apply_morphological_operation(image)
    elif selected_option == 'Contour Detection':
        return detect_edge_contours(image)[0]
    elif selected_option == 'Corner Points':
        return find_corner_points(image)[0]
    elif selected_option == 'Perspective Transform':
        return apply_perspective_transform(image)
    else:
        return image

def main():
     # Set title and page configuration
    st.set_page_config(page_title='Document Scanner', layout='centered', initial_sidebar_state='expanded')

    # Add a title and description
    st.title('Document Scanner')
    st.write("Upload an image and select a processing technique from the dropdown menu.")


    uploaded_file = st.file_uploader("Choose a file", type=['jpg', 'png', 'jpeg'])

    if uploaded_file is not None:
        image = Image.open(uploaded_file)
        dim_limit = 1080
        max_dim = max(image.size)
        if max_dim > dim_limit:
            resize_scale = dim_limit / max_dim
            image = image.resize((int(image.width * resize_scale), int(image.height * resize_scale)))

        st.image(image, caption='Uploaded Image', use_column_width=True)

        selected_option = st.selectbox('Select Image Processing Technique', [
            'Original',
            'Morphological Operation',
            'Contour Detection',
            'Corner Points',
            'Perspective Transform'
        ])

        if selected_option != 'Original':
            image_array = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)
            processed_image = process_image(image_array, selected_option)
            st.image(processed_image, caption=f'{selected_option} Result', use_column_width=True)

        translate_button = st.button("Translate to Hebrew")
        if translate_button:
              translated_text = perform_ocr_and_translate(image)
              st.write("Translated Text:")
              st.write(translated_text)


        # Add a footer
        st.markdown(
            "---\nMade with ❤️ by Elyasaf Cohen",
            unsafe_allow_html=True
        )


if __name__ == '__main__':
    main()


Overwriting app.py


In [29]:
!npm install localtunnel

npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ localtunnel@2.0.2
updated 1 package and audited 36 packages in 0.512s

3 packages are looking for funding
  run `npm fund` for details

found 2 moderate severity vulnerabilities
  run `npm audit fix` to fix them, or `npm audit` for details


In [30]:
import urllib
print("Password/Enpoint IP for localtunnel is:",urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))

Password/Enpoint IP for localtunnel is: 34.29.200.114


In [31]:
!streamlit run app.py &>/content/logs.txt &

In [32]:
!npx localtunnel --port 8501 &

npx: installed 22 in 1.928s
your url is: https://sad-baths-yell.loca.lt
